In [10]:
import os
import mmcv
import torch
import albumentations as A

from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
from pycocotools.coco import COCO
import pickle
import pandas as pd
import numpy as np
import json

In [2]:
# epoch 지정해주기
epoch = 'best_mIoU_epoch_36'
# work dir 설정해주기
work_dir = '/opt/ml/mmsegmentation/work_dirs/upernet_convnext_xlarge_fp16_640x640_160k_ade20k_pseudo_offline/'

# config file 들고오기
# config file은 수정해서 사용
cfg = Config.fromfile('/opt/ml/mmsegmentation/configs/convnext/upernet_convnext_xlarge_fp16_640x640_160k_ade20k_pseudo_offline.py')
root='/opt/ml/input/mmseg/images/test'

# dataset config 수정
cfg.data.test.img_dir = root
# cfg.data.test.pipeline[1]['img_scale'] = (512,512)
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1
cfg.work_dir = work_dir

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [3]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=4,
        dist=False,
        shuffle=False)

2022-05-12 09:10:51,762 - mmseg - INFO - Loaded 624 images


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:232: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: /opt/ml/mmsegmentation/work_dirs/upernet_convnext_xlarge_fp16_640x640_160k_ade20k_pseudo_offline/best_mIoU_epoch_36.pth


In [5]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 624/624, 0.3 task/s, elapsed: 2344s, ETA:     0s